In [4]:
import pandas as pd
import numpy as np
import re 
from tqdm import tqdm

import json
import requests
from ast import literal_eval

In [5]:
class LocalData_load :
    def __init__(self) :
        self.local_api_url = "https://bigdata.kepco.co.kr/openapi/v1/powerUsage/industryType.do?"
        self.api_key = "7aR4rW85162N8QZc1kO9G8un6i3HBGuihtA5M007"

    ## sort raw data into DataFrames
    def search(self, start_year, start_mon, end_year, end_mon) :
        super(LocalData_load).__init__()
        
        power_use = pd.DataFrame(columns=['year', 'month', 'city', 'biz', 'powerUseage'])
        
        for year in tqdm(range(start_year, end_year+1, 1)):
            for month in range(start_mon, end_mon+1, 1):
                if month < 10: 
                    params = {"apiKey" : self.api_key, "year" : str(year), "month" : "0"+str(month)}
                else: 
                    params = {"apiKey" : self.api_key, "year" : str(year), "month" : str(month)}
                response = requests.get(self.local_api_url, params = params)
                content = response.content.decode("utf-8")
        
                df = pd.DataFrame(None)
                df['year'] = [re.sub("[^0-9]", "", row.split(":")[1]) for row in re.findall('"year":.+?[0-9]{4}', content)]
                df['month'] = [re.sub("[^0-9]", "", row.split(":")[1]) for row in re.findall('"month":.+?[0-9]{2}', content)]
                df['city'] = [re.sub("[^가-힣]", "", row.split(":")[1]) for row in re.findall('"city":.+?[가-힣]{2,}', content)]
                df['biz'] = [re.sub("[^가-힣, ]", "", row.split(":")[1]) for row in re.findall('"biz":.+?[가-힣, ]{2,}', content)]
                df['powerUseage'] = [re.sub("[^0-9]", "", row.split(":")[1]) for row in re.findall('"powerUsage":.+?[0-9]{1,}', content)]
                power_use = pd.concat([power_use,df])
                
        power_use = power_use[power_use['city']=='전체']
        power_use['powerUseage'] = power_use['powerUseage'].astype(float)/1000
        power_use.reset_index(drop=True, inplace=True)
        return power_use
    
    ## Total power usage in total, manufacturing and service industries
    def total_useage(self, start_year, start_mon, end_year, end_mon):
        super(LocalData_load).__init__()
        df = func.search(start_year, start_mon, end_year, end_mon) # sort raw data into DataFrames

        total =  pd.DataFrame(df.groupby(['year', 'month'])['powerUseage'].sum()).reset_index()
        service_dic = df[df['biz'].isin(['하수폐기처리원료재생환경복원', '도매 및 소매업', '운수업', '숙박 및 음식점업 ', 
                                           '출판영상,방송통신정보서비스업', '금융 및 보험업 ', '부동산업 및 임대업 ', '전문, 과학 및 기술 서비스업 ', 
                                            '사업시설관리,사업지원서비스업', '공공행정, 국방 및 사회보장 행정', '교육 서비스업', 
                                           '보건업 및 사회복지 서비스업', '예술,스포츠및여가관련서비스업', '협회단체,수리기타개인서비스업', 
                                           '가구내고용,미분류자가소비생산', '국제 및 외국기관'])]
        sevice = pd.DataFrame(service_dic.groupby(['year', 'month'])['powerUseage'].sum()).reset_index()

        total['manufact'] = df[df['biz']=='제조업 '].reset_index(drop=True)['powerUseage']
        total['sevice'] = sevice['powerUseage']
        total['datetime'] = pd.to_datetime(total.year + " " + total.month, format='%Y-%m')
        total['datetime'] = total['datetime'].astype(str).str.slice(stop=7)
        total = total.set_index('datetime')
        total.drop(['year', 'month'], axis=1, inplace=True)
        total = total.rename(columns={'powerUseage':'total'})
        
        
        return total

In [8]:
pd.options.display.float_format = '{:.3f}'.format # without exponential 

func = LocalData_load()
# df = func.search(2017, 1, 2018, 11) #2017
sum_data = func.total_useage(2017, 1, 2018, 11) # start year, start month, end year, end month

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:43<00:00, 21.92s/it]


In [9]:
sum_data

,total,manufact,sevice
datetime,,,
2017-07,36224735.007,22128019.531,12046504.745
2017-08,37114897.111,21708908.958,13130541.747
2017-09,35409569.060,21727256.670,11512808.278
2017-10,32053912.762,20408028.073,9704694.947
2017-11,34129114.227,21371666.629,10230255.380
2018-01,39182016.481,22609240.498,13657016.259
2018-02,37396605.395,20750132.814,13687326.263
2018-03,35159905.557,21455705.903,11231176.938
2018-04,34570017.788,21821984.888,10494127.221


In [90]:
sum_data

,total,manufact,sevice
datetime,,,
2017-07,36224735.007,22128019.531,12046504.745
2017-08,37114897.111,21708908.958,13130541.747
2017-09,35409569.060,21727256.670,11512808.278
2017-10,32053912.762,20408028.073,9704694.947
2017-11,34129114.227,21371666.629,10230255.380


In [87]:
sum_data1

,total,manufact,sevice
datetime,,,
2017-07,36224735007.000,22128019531.000,12046504745.000
2017-08,37114897111.000,21708908958.000,13130541747.000
2017-09,35409569060.000,21727256670.000,11512808278.000
2017-10,32053912762.000,20408028073.000,9704694947.000
2017-11,34129114227.000,21371666629.000,10230255380.000
